In [5]:
# install pip dependencies
! pip install kagglehub
! pip install pandas
! pip install torch datasets
! pip install ipdb

In [6]:
# kaggle authentication
# upload kaggle.json
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

The syntax of the command is incorrect.


'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
# import libraries
import kagglehub
import os
import pandas as pd
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import gc
import numpy as np
from tqdm import tqdm
import io
import tarfile

c:\Users\imlam\Desktop\connectify\ids-hackathon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# download data to paraquet
download_path = kagglehub.dataset_download("solarmainframe/ids-intrusion-csv")
raw_csv_files = glob.glob(os.path.join(download_path, "*.csv"))
print(raw_csv_files)

100%|██████████| 1.60G/1.60G [02:13<00:00, 12.9MB/s]

Extracting files...


['C:\\Users\\imlam\\.cache\\kagglehub\\datasets\\solarmainframe\\ids-intrusion-csv\\versions\\1\\02-14-2018.csv', 'C:\\Users\\imlam\\.cache\\kagglehub\\datasets\\solarmainframe\\ids-intrusion-csv\\versions\\1\\02-15-2018.csv', 'C:\\Users\\imlam\\.cache\\kagglehub\\datasets\\solarmainframe\\ids-intrusion-csv\\versions\\1\\02-16-2018.csv', 'C:\\Users\\imlam\\.cache\\kagglehub\\datasets\\solarmainframe\\ids-intrusion-csv\\versions\\1\\02-20-2018.csv', 'C:\\Users\\imlam\\.cache\\kagglehub\\datasets\\solarmainframe\\ids-intrusion-csv\\versions\\1\\02-21-2018.csv', 'C:\\Users\\imlam\\.cache\\kagglehub\\datasets\\solarmainframe\\ids-intrusion-csv\\versions\\1\\02-22-2018.csv', 'C:\\Users\\imlam\\.cache\\kagglehub\\datasets\\solarmainframe\\ids-intrusion-csv\\versions\\1\\02-23-2018.csv', 'C:\\Users\\imlam\\.cache\\kagglehub\\datasets\\solarmainframe\\ids-intrusion-csv\\versions\\1\\02-28-2018.csv', 'C:\\Users\\imlam\\.cache\\kagglehub\\datasets\\solarmainframe\\ids-intrusion-csv\\versions\\1\

In [9]:
# split csvs into managable chunks

CSV_FILES = []

num_raw_files = len(raw_csv_files)
for i, file in enumerate(raw_csv_files):
  print(f"splitting: {i+1}/{num_raw_files}")
  filename = os.path.splitext(os.path.basename(file))[0]
  max_bytes = 500 * 1024 * 1024

  part_idx = 1
  current_size = 0

  with open(file, 'r') as infile:
    header = infile.readline()
    lines_buffer = []
    for line in infile:
      lines_buffer.append(line)
      current_size += len(line.encode('utf-8'))  # size in bytes

      # If current chunk exceeds the limit, write to file
      if current_size >= max_bytes:
        out_path = os.path.join(f"{filename}_part{part_idx}.csv")
        with open(out_path, 'w') as outfile:
          outfile.write(header)
          outfile.writelines(lines_buffer)
        CSV_FILES.append(out_path)
        part_idx += 1
        lines_buffer = []
        current_size = 0

    # Write remaining lines
    if lines_buffer:
      out_path = os.path.join(f"{filename}_part{part_idx}.csv")
      with open(out_path, 'w') as outfile:
        outfile.write(header)
        outfile.writelines(lines_buffer)
      CSV_FILES.append(out_path)

splitting: 1/10
splitting: 2/10
splitting: 3/10
splitting: 4/10
splitting: 5/10
splitting: 6/10
splitting: 7/10
splitting: 8/10
splitting: 9/10
splitting: 10/10


In [10]:
test_df = pd.read_csv(CSV_FILES[1])
NUM_COLUMNS = len(test_df.columns)
print(test_df.columns)
del test_df
gc.collect()

Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
      

39

In [11]:
# canon columns
CANON_COLUMN_INDEX = ['Fwd IAT Tot', 'Fwd Pkt Len Min', 'Down/Up Ratio', 'Dst Port', 'Fwd IAT Std', 'Fwd Header Len', 'Fwd IAT Min', 'Flow IAT Std', 'Active Std', 'Bwd IAT Max', 'Fwd Pkt Len Mean', 'Pkt Size Avg', 'PSH Flag Cnt', 'Flow IAT Mean', 'Fwd Act Data Pkts', 'Bwd Pkt Len Max', 'Flow IAT Max', 'ACK Flag Cnt', 'Bwd IAT Tot', 'Flow IAT Min', 'Bwd Pkts/b Avg', 'Fwd IAT Max', 'SYN Flag Cnt', 'Bwd Header Len', 'Fwd Seg Size Avg', 'Bwd Byts/b Avg', 'Subflow Bwd Byts', 'Pkt Len Max', 'Bwd Pkts/s', 'Fwd IAT Mean', 'Pkt Len Var', 'Fwd Pkt Len Std', 'Protocol', 'Init Bwd Win Byts', 'Active Min', 'Src Port', 'RST Flag Cnt', 'Subflow Fwd Byts', 'Init Fwd Win Byts', 'Bwd Pkt Len Std', 'Fwd PSH Flags', 'Fwd Pkts/s', 'Bwd Blk Rate Avg', 'Flow Byts/s', 'CWE Flag Count', 'Pkt Len Std', 'Active Max', 'Fwd Byts/b Avg', 'Fwd Blk Rate Avg', 'URG Flag Cnt', 'Timestamp', 'Fwd Pkts/b Avg', 'Idle Mean', 'Idle Std', 'Fwd Pkt Len Max', 'Pkt Len Min', 'Flow Duration', 'Fwd Seg Size Min', 'Bwd IAT Min', 'TotLen Fwd Pkts', 'Flow Pkts/s', 'Active Mean', 'ECE Flag Cnt', 'Idle Min', 'Subflow Bwd Pkts', 'Bwd Pkt Len Mean', 'Pkt Len Mean', 'Tot Fwd Pkts', 'Bwd IAT Std', 'Bwd Seg Size Avg', 'Bwd URG Flags', 'Bwd Pkt Len Min', 'Tot Bwd Pkts', 'Subflow Fwd Pkts', 'Bwd IAT Mean', 'FIN Flag Cnt', 'Bwd PSH Flags', 'TotLen Bwd Pkts', 'Fwd URG Flags', 'Idle Max']
CANON_COLUMN_INDEX.sort()
CANON_COLUMN_INDEX.append('Label')
print(CANON_COLUMN_INDEX)
TRAINING_UNWANTED_COLUMNS = ['Timestamp', 'Flow ID', 'Dst IP', "Src IP"]

['ACK Flag Cnt', 'Active Max', 'Active Mean', 'Active Min', 'Active Std', 'Bwd Blk Rate Avg', 'Bwd Byts/b Avg', 'Bwd Header Len', 'Bwd IAT Max', 'Bwd IAT Mean', 'Bwd IAT Min', 'Bwd IAT Std', 'Bwd IAT Tot', 'Bwd PSH Flags', 'Bwd Pkt Len Max', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Min', 'Bwd Pkt Len Std', 'Bwd Pkts/b Avg', 'Bwd Pkts/s', 'Bwd Seg Size Avg', 'Bwd URG Flags', 'CWE Flag Count', 'Down/Up Ratio', 'Dst Port', 'ECE Flag Cnt', 'FIN Flag Cnt', 'Flow Byts/s', 'Flow Duration', 'Flow IAT Max', 'Flow IAT Mean', 'Flow IAT Min', 'Flow IAT Std', 'Flow Pkts/s', 'Fwd Act Data Pkts', 'Fwd Blk Rate Avg', 'Fwd Byts/b Avg', 'Fwd Header Len', 'Fwd IAT Max', 'Fwd IAT Mean', 'Fwd IAT Min', 'Fwd IAT Std', 'Fwd IAT Tot', 'Fwd PSH Flags', 'Fwd Pkt Len Max', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Min', 'Fwd Pkt Len Std', 'Fwd Pkts/b Avg', 'Fwd Pkts/s', 'Fwd Seg Size Avg', 'Fwd Seg Size Min', 'Fwd URG Flags', 'Idle Max', 'Idle Mean', 'Idle Min', 'Idle Std', 'Init Bwd Win Byts', 'Init Fwd Win Byts', 'PSH Flag Cnt

In [12]:
# get normalization stats
UNWANTED_COLUMNS_STATS = ['Timestamp','Label', 'Flow ID', 'Dst IP', "Src IP"]

def calcStatistics(files):
  n_total = 0
  mean_total = pd.Series(0.0)
  M2 = pd.Series(0.0)

  file_idx = 0
  num_files = len(files)
  for file in files:
    print(f"calculating: {file_idx+1}/{num_files}")
    df = pd.read_csv(file)
    df = df.reindex(columns=CANON_COLUMN_INDEX)
    for col in df.columns:
      if col not in UNWANTED_COLUMNS_STATS:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('float32')

        if col not in mean_total:
          mean_total[col] = 0.0
          M2[col] = 0.0

      else:
        df = df.drop(columns=[col])

    n = len(df)
    mean = df.mean()
    var = df.var(ddof=0)

    # Welford’s algorithm for combining means and variances
    delta = mean - mean_total
    new_n = n_total + n

    mean_total = (n_total * mean_total + n * mean) / new_n
    M2 += var * n + (delta**2) * n_total * n / new_n

    n_total = new_n

    file_idx += 1
    del df
    gc.collect()

  std = (M2 / n_total) ** 0.5
  length = n_total
  mean = mean_total
  return (length, mean, std)

LENGTH, MEAN, STD = calcStatistics(CSV_FILES)

import json
j = {'length': LENGTH, 'mean': MEAN.to_dict(), 'std': STD.to_dict(), 'non-stat-columns': UNWANTED_COLUMNS_STATS}
with open('info.json', 'w') as f:
  json.dump(j,f)

calculating: 1/17


c:\Users\imlam\Desktop\connectify\ids-hackathon\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


calculating: 2/17


c:\Users\imlam\Desktop\connectify\ids-hackathon\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


calculating: 3/17


...

C:\Users\imlam\AppData\Local\Temp\ipykernel_27584\1814859186.py:13: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
c:\Users\imlam\Desktop\connectify\ids-hackathon\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


calculating: 16/17


C:\Users\imlam\AppData\Local\Temp\ipykernel_27584\1814859186.py:13: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
c:\Users\imlam\Desktop\connectify\ids-hackathon\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


calculating: 17/17


c:\Users\imlam\Desktop\connectify\ids-hackathon\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [16]:
# normalize dataset and convert to parquet
! mkdir "normalized-benign/"

UNWANTED_COLUMNS_NORM = ['Flow ID', 'Dst IP', "Src IP"]

def normalize(files, mean, std):
  pq_files = []

  num_files = len(files)
  for i,file in enumerate(CSV_FILES):
    print(f"normalizing: {i+1}/{num_files}")
    df = pd.read_csv(file)
    df = df.reindex(columns=CANON_COLUMN_INDEX)
    filename = os.path.splitext(os.path.basename(file))[0]
    name = f'normalized-benign/normalized_{filename}-benign.parquet'
    pq_files.append(name)

    for col in df.columns:
      if col not in UNWANTED_COLUMNS_STATS:
          df[col] = pd.to_numeric(df[col], errors='coerce').astype('float32')
          df[col] = (df[col] - mean[col]) / std[col]
      else:
        if col in UNWANTED_COLUMNS_NORM:
          df = df.drop(columns=[col])

    if 'Src Port' not in df.columns:
      df['Src Port'] = np.nan

    df = df[df['Label'] == "Benign"]

    df.to_parquet(name)

    del df
    gc.collect()
  return pq_files

PARQUET_FILES = normalize(CSV_FILES, MEAN, STD)
print(PARQUET_FILES)

A subdirectory or file normalized/ already exists.


normalizing: 1/17
normalizing: 2/17
normalizing: 3/17


C:\Users\imlam\AppData\Local\Temp\ipykernel_27584\3035799491.py:12: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


normalizing: 4/17
normalizing: 5/17
normalizing: 6/17
normalizing: 7/17
normalizing: 8/17
normalizing: 9/17
normalizing: 10/17
normalizing: 11/17
normalizing: 12/17
normalizing: 13/17
normalizing: 14/17
normalizing: 15/17


C:\Users\imlam\AppData\Local\Temp\ipykernel_27584\3035799491.py:12: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


normalizing: 16/17


C:\Users\imlam\AppData\Local\Temp\ipykernel_27584\3035799491.py:12: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


normalizing: 17/17
['normalized-benign/normalized_02-14-2018_part1-benign.parquet', 'normalized-benign/normalized_02-15-2018_part1-benign.parquet', 'normalized-benign/normalized_02-16-2018_part1-benign.parquet', 'normalized-benign/normalized_02-20-2018_part1-benign.parquet', 'normalized-benign/normalized_02-20-2018_part2-benign.parquet', 'normalized-benign/normalized_02-20-2018_part3-benign.parquet', 'normalized-benign/normalized_02-20-2018_part4-benign.parquet', 'normalized-benign/normalized_02-20-2018_part5-benign.parquet', 'normalized-benign/normalized_02-20-2018_part6-benign.parquet', 'normalized-benign/normalized_02-20-2018_part7-benign.parquet', 'normalized-benign/normalized_02-20-2018_part8-benign.parquet', 'normalized-benign/normalized_02-21-2018_part1-benign.parquet', 'normalized-benign/normalized_02-22-2018_part1-benign.parquet', 'normalized-benign/normalized_02-23-2018_part1-benign.parquet', 'normalized-benign/normalized_02-28-2018_part1-benign.parquet', 'normalized-benign/n

In [17]:
# double check columns
base_df = pd.read_parquet(PARQUET_FILES[0])
base = set(list(base_df.columns))
del base_df
gc.collect()

for f in PARQUET_FILES:
  df = pd.read_parquet(f)

  print(df[df['Label'] != "Benign"])

  cols = set(list(df.columns))
  if base != cols:
    print(f'in base: {base - cols}, in cols: {cols - base}')
  del df
  gc.collect()

Empty DataFrame
Columns: [ACK Flag Cnt, Active Max, Active Mean, Active Min, Active Std, Bwd Blk Rate Avg, Bwd Byts/b Avg, Bwd Header Len, Bwd IAT Max, Bwd IAT Mean, Bwd IAT Min, Bwd IAT Std, Bwd IAT Tot, Bwd PSH Flags, Bwd Pkt Len Max, Bwd Pkt Len Mean, Bwd Pkt Len Min, Bwd Pkt Len Std, Bwd Pkts/b Avg, Bwd Pkts/s, Bwd Seg Size Avg, Bwd URG Flags, CWE Flag Count, Down/Up Ratio, Dst Port, ECE Flag Cnt, FIN Flag Cnt, Flow Byts/s, Flow Duration, Flow IAT Max, Flow IAT Mean, Flow IAT Min, Flow IAT Std, Flow Pkts/s, Fwd Act Data Pkts, Fwd Blk Rate Avg, Fwd Byts/b Avg, Fwd Header Len, Fwd IAT Max, Fwd IAT Mean, Fwd IAT Min, Fwd IAT Std, Fwd IAT Tot, Fwd PSH Flags, Fwd Pkt Len Max, Fwd Pkt Len Mean, Fwd Pkt Len Min, Fwd Pkt Len Std, Fwd Pkts/b Avg, Fwd Pkts/s, Fwd Seg Size Avg, Fwd Seg Size Min, Fwd URG Flags, Idle Max, Idle Mean, Idle Min, Idle Std, Init Bwd Win Byts, Init Fwd Win Byts, PSH Flag Cnt, Pkt Len Max, Pkt Len Mean, Pkt Len Min, Pkt Len Std, Pkt Len Var, Pkt Size Avg, Protocol, RS

In [22]:
# from google.colab import drive
# drive.mount('/content/drive')
! tar -czvf normalized-benign-ids2018-parquet.tar.gz normalized-benign/*
# ! cp normalized-ids2018.tar.gz drive/MyDrive/

a normalized-benign/info.json
a normalized-benign/normalized_02-14-2018_part1-benign.parquet
a normalized-benign/normalized_02-15-2018_part1-benign.parquet
a normalized-benign/normalized_02-16-2018_part1-benign.parquet
a normalized-benign/normalized_02-20-2018_part1-benign.parquet
a normalized-benign/normalized_02-20-2018_part2-benign.parquet
a normalized-benign/normalized_02-20-2018_part3-benign.parquet
a normalized-benign/normalized_02-20-2018_part4-benign.parquet
a normalized-benign/normalized_02-20-2018_part5-benign.parquet
a normalized-benign/normalized_02-20-2018_part6-benign.parquet
a normalized-benign/normalized_02-20-2018_part7-benign.parquet
a normalized-benign/normalized_02-20-2018_part8-benign.parquet
a normalized-benign/normalized_02-21-2018_part1-benign.parquet
a normalized-benign/normalized_02-22-2018_part1-benign.parquet
a normalized-benign/normalized_02-23-2018_part1-benign.parquet
a normalized-benign/normalized_02-28-2018_part1-benign.parquet
a normalized-benign/norma